<img src="https://www.dropbox.com/s/l93ecugg1jbtjiw/fga.png?raw=1" width="1200" height="400">
<ul>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Primeira (1/3) Atividade Prática Avaliativa
    Entrega da Atividade 27/07/2022 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol> 
<li> Colete os preços e volumes de um conjunto de ativos durante o período de 10 (quinze) anos, pertencentes a no mínimo 5 (cinco) Índices de Segmentos e Setoriais diferentes na B3, e separe os 30 (cinquenta) ativos mais líquidos; </li><p></p>
    <li> Construa amostras de treinamento e outras de teste, sendo a primeira com os 3 (três) anos e a segunda com o último ano, e desloque as janelas anualmente (walk forward analysis); </li> <p></p>
    <li> Na amostra de treinamento, aplique uma clusterização não supervisionada (K-Means ou DBSCAN) e separe um ativo para cada clusters, com o critério que desejar (sugestão maior índice Sharpe); </li><p></p>
    <li> Aplique a Teoria do Portfolio Moderno de Markowitz nos ativos escolhidos no passo anterior, no sentido de alocar um capital de R$100.000,00; </li><p></p>
    <li> Verifique o resultado da alocação de capital com Markowitz nos ativos separados, na fase de treinamento, na amostra de teste que representa o último ano da aquisição e compare com o índice Bovespa; </li><p></p>
    <li> Descreva suas impressores relativas ao processo, baseado em números, e resultados encontrados. </li>
</ol>
</div>


### Questão 1: 

In [2]:
# !pip install pandas yfinance tqdm seaborn ipywidgets
import pandas as pd
#import investpy
import yfinance as yf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta

In [3]:
end_date = dt.date.today()
start_date = end_date  - relativedelta(years=10)

print(f'start_date = {start_date}')
print(f'end_date = {end_date}')

start_date = 2012-07-27
end_date = 2022-07-27


In [4]:
financial_assets = list([])

with open('financial_assets.txt', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        financial_assets.append(line.rstrip('\n') + '.SA')

df = pd.DataFrame(financial_assets, columns=['ATIVOS'])

df.tail()

,ATIVOS
30,BRFS3.SA
31,JBSS3.SA
32,CCRO3.SA
33,RAIL3.SA
34,CMIN3.SA


In [5]:
def filter_financial_assets(financial_assets: list = []) -> list:
    """
    Filter for 30 financial assets with greater volume 
    """
    # Download in yahoo finance
    list_pandas_data_frame: list = []
    
    for financial_asset in financial_assets:
        list_pandas_data_frame.append(yf.download(financial_asset, start=str(start_date), end=str(end_date))) 
        
    # Get values of volume columns
    list_volumes: list = []
    list_prices: list = []
    
    for financial_asset_df in list_pandas_data_frame:
        list_volumes.append(sum(financial_asset_df['Volume'].tolist()))
    
    # For debug, object key = ativo value = somatorio_do_volume
    financial_asset_and_volume = dict(zip(financial_assets, list_volumes))
    
    # Get 30 greater ativos with greater volumes
    list_30_financial_asset_greater_volumes: list =[]
    list_volume_for_filter = list_volumes.copy()
    
    for i in range(30):
        max_volume = max(list_volume_for_filter)
        list_30_financial_asset_greater_volumes.append(financial_assets[list_volume_for_filter.index(max_volume)])
        list_volume_for_filter.remove(max_volume)
    
    # Get 30 ativos with price and volume
    list_financial_asset_with_price_and_volume: list = []
    columns_ignored = ['Open','Low', 'High','Adj Close']
    
    for i in range(len(list_30_financial_asset_greater_volumes)):
        list_financial_asset_with_price_and_volume.append(yf.download(list_30_financial_asset_greater_volumes[i], start=str(start_date), end=str(end_date)).drop(columns=columns_ignored))
        
    return list_financial_asset_with_price_and_volume


# Tests filter
filter_financial_assets(financial_assets)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[               Close     Volume
 Date                           
 2012-07-27  0.310937   11664000
 2012-07-30  0.315312    5849600
 2012-07-31  0.320312    7120000
 2012-08-01  0.328750   12812800
 2012-08-02  0.326875    9699200
 ...              ...        ...
 2022-07-20  3.070000  224980300
 2022-07-21  3.010000  165905300
 2022-07-22  2.860000  161429800
 2022-07-25  2.790000  129819500
 2022-07-26  2.610000  139841400
 
 [2480 rows x 2 columns],
             Close    Volume
 Date                       
 2012-07-27   4.55         0
 2012-07-30   4.55         0
 2012-07-31   4.55         0
 2012-08-01   4.55         0
 2012-08-02   4.55         0
 ...           ...       ...
 2022-07-20   2.61  82037100
 2022-07-21   2.63  49132400
 2022-07-22   2.51  47745600
 2022-07-25   2.52  32139800
 2022-07-26   2.36  40297300
 
 [2480 rows x 2 columns],
             Close    Volume
 Date                       
 2012-07-27   4.55         0
 2012-07-30   4.55         0
 2012-07-31   4.55    

In [8]:
a = [12,546,7,7,6,8,9]
a[:4]

[12, 546, 7, 7]

### Questão 2: 

### Questão 3: 

### Questão 4: 

### Questão 5: 

### Questão 6: 